In [1]:
from pymatgen.ext.matproj import MPRester

API_KEY = "Yoa1b2uiwwxd5fpoSFS9aaTg7qSuvnF1"  # Replace with your real key
mpr = MPRester(API_KEY)

# List of MP material IDs
mp_ids = ["mp-1960", "mp-841", "mp-942733", "mp-2858", "mp-1968"]

# Query summaries
summaries = mpr.summary.search(material_ids=mp_ids)

# Print formation energies per atom
for s in summaries:
    print(f"{s.material_id:<12} {s.formula_pretty:<20} Formation Energy (eV/atom): {s.formation_energy_per_atom:.6f}")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1681958/3682464946.py:10: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  summaries = mpr.summary.search(material_ids=mp_ids)
Retrieving SummaryDoc documents: 100%|██████████| 5/5 [00:00<00:00, 38550.59it/s]

mp-841       Li2O2                Formation Energy (eV/atom): -1.650170
mp-1960      Li2O                 Formation Energy (eV/atom): -2.061598
mp-942733    Li7La3Zr2O12         Formation Energy (eV/atom): -3.124117
mp-2858      ZrO2                 Formation Energy (eV/atom): -3.813618
mp-1968      La2O3                Formation Energy (eV/atom): -3.875929


In [11]:
import os
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ---- 1. CHGNet model -------------------------------------------------
model = CHGNet.load()   # e_pred is eV/atom !

# ---- 2. elemental chemical potentials (CHGNet column) ----------------
mu = {"Li": -1.882, "La": -4.894, "Zr": -8.509, "O": -4.913}

# ---- 3. CIFs ----------------------------------------------------------
cif_dir = "./cifs"
files = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

for fname, label in files.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    n_atoms = struct.composition.num_atoms
    
    # ------- CHGNet prediction (already per atom) ----------------------
    e_pred_atom = model.predict_structure(struct)["e"]
    
    # ------- reference energy per atom ---------------------------------
    ref_per_atom = sum(struct.composition[el] * mu[str(el)] for el in struct.elements) / n_atoms
    
    # ------- formation energy per atom ---------------------------------
    e_form = e_pred_atom - ref_per_atom
    
    print(f"{label:15s}:  E_form (CHGNet) = {e_form: .6f} eV/atom")


CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Li2O2          :  E_form (CHGNet) = -1.627611 eV/atom
Li2O           :  E_form (CHGNet) = -2.034650 eV/atom
Li7La3Zr2O12   :  E_form (CHGNet) = -3.121807 eV/atom
ZrO2           :  E_form (CHGNet) = -3.813102 eV/atom
La2O3          :  E_form (CHGNet) = -3.871564 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [ ]:
import os
import torch
from pymatgen.core import Structure
from ase import Atoms
from pymatgen.io.ase import AseAtomsAdaptor
from mace.calculators import MACECalculator

# ---- 1. Load the MACE model -------------------------------------------
model_path = "2024-07-12-mace-128-L1_epoch-199.model"
mace_calc = MACECalculator(model_paths=model_path, device='cuda')

# ---- 2. Reference μ_model from MACE -----------------------------------
mu_mace = {
    "Li": -1.884929,
    "La": -4.898304,
    "Zr": -8.523547,
    "O":  -4.850042,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE) = {e_form: .6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices = torch.l

cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE) = -1.351434 eV/atom
Li2O           :  E_form (MACE) = -1.854707 eV/atom
Li7La3Zr2O12   :  E_form (MACE) = -2.825235 eV/atom
ZrO2           :  E_form (MACE) = -3.395213 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


La2O3          :  E_form (MACE) = -3.504901 eV/atom


In [14]:
import pandas as pd

# Energies
data = {
    "Compound": ["Li2O2", "Li2O", "Li7La3Zr2O12", "ZrO2", "La2O3"],
    "E_form_MP":     [-1.65017, -2.06160, -3.12412, -3.81362, -3.87593],
    "E_form_CHGNet": [-1.6276,  -2.0347,  -3.1218,  -3.8131,  -3.8716],
    "E_form_MACE":   [-1.3514,  -1.8547,  -2.8252,  -3.3952,  -3.5049],
}

df = pd.DataFrame(data)

# Errors
df["CHGNet_Error"] = df["E_form_CHGNet"] - df["E_form_MP"]
df["MACE_Error"] = df["E_form_MACE"] - df["E_form_MP"]

# Save to Excel
df.to_excel("final_formation_energy_with_errors.xlsx", index=False)

# Print to confirm
print(df)


       Compound  E_form_MP  E_form_CHGNet  E_form_MACE  CHGNet_Error  \
0         Li2O2   -1.65017        -1.6276      -1.3514       0.02257   
1          Li2O   -2.06160        -2.0347      -1.8547       0.02690   
2  Li7La3Zr2O12   -3.12412        -3.1218      -2.8252       0.00232   
3          ZrO2   -3.81362        -3.8131      -3.3952       0.00052   
4         La2O3   -3.87593        -3.8716      -3.5049       0.00433   

   MACE_Error  
0     0.29877  
1     0.20690  
2     0.29892  
3     0.41842  
4     0.37103  


## Multiple MACE models comparision
We first go with MACE-MP-0a	medium.
1. Chemical potentials predictions using cif files ->

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2023-12-03-mace-128-L1_epoch-199.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("Li.cif") 
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.905607 eV/atom
La: μ_model = -4.902058 eV/atom
Zr: μ_model = -8.530130 eV/atom
O: μ_model = -4.925096 eV/atom


Now we predict the formation energies

In [ ]:
import os
import torch
from pymatgen.core import Structure
from ase import Atoms
from pymatgen.io.ase import AseAtomsAdaptor
from mace.calculators import MACECalculator

# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2023-12-03-mace-128-L1_epoch-199.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
mu_mace = {
    "Li": -1.905607,
    "La": -4.902058,
    "Zr": -8.530130,
    "O":  -4.925096 ,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MP-0a-medium) = {e_form: .6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE-MP-0a-medium) = -1.366339 eV/atom
Li2O           :  E_form (MACE-MP-0a-medium) = -1.832991 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MP-0a-medium) = -2.791143 eV/atom
ZrO2           :  E_form (MACE-MP-0a-medium) = -3.354554 eV/atom
La2O3          :  E_form (MACE-MP-0a-medium) = -3.467417 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


## MACE-MP-0a Small
1. Chemical potentials  

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2023-12-10-mace-128-L0_energy_epoch-249.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.905902 eV/atom
La: μ_model = -4.919243 eV/atom
Zr: μ_model = -8.594373 eV/atom
O: μ_model = -4.930625 eV/atom


2. Now comes the formation energy part

In [ ]:
# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2023-12-10-mace-128-L0_energy_epoch-249.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
mu_mace = {
    "Li": -1.905902,
    "La": -4.919243,
    "Zr": -8.594373,
    "O":  -4.930625 ,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MP-0a-small) = {e_form: .6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE-MP-0a-small) = -1.378085 eV/atom
Li2O           :  E_form (MACE-MP-0a-small) = -1.828398 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MP-0a-small) = -2.787109 eV/atom
ZrO2           :  E_form (MACE-MP-0a-small) = -3.339057 eV/atom
La2O3          :  E_form (MACE-MP-0a-small) = -3.450940 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Now we do for MACE-MP-0a-large
1. Chemical potentials

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2024-01-07-mace-128-L2_epoch-199.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.894110 eV/atom
La: μ_model = -4.891070 eV/atom
Zr: μ_model = -8.539969 eV/atom
O: μ_model = -4.870034 eV/atom


2. Formation energies

In [ ]:
# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/2023-12-10-mace-128-L0_energy_epoch-249.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": -1.894110,
    "La": -4.891070,
    "Zr": -8.539969,
    "O":  -4.870034,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MP-0a-large) = {e_form: .6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE-MP-0a-large) = -1.414277 eV/atom
Li2O           :  E_form (MACE-MP-0a-large) = -1.856457 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MP-0a-large) = -2.828900 eV/atom
ZrO2           :  E_form (MACE-MP-0a-large) = -3.397586 eV/atom
La2O3          :  E_form (MACE-MP-0a-large) = -3.498564 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


## MACE-mp-0b3-medium
1. Chemical potential

In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/mace-mp-0b3-medium.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.906338 eV/atom
La: μ_model = -4.895953 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


Zr: μ_model = -8.559929 eV/atom
O: μ_model = -4.901506 eV/atom


2. Formn energies

In [ ]:
# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/mehuldarak/MACE_models/universal_09072025/mace-mp-0b3-medium.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": -1.894110,
    "La": -4.891070,
    "Zr": -8.539969,
    "O":  -4.870034,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (mace-mp-0b3-medium) = {e_form: .6f} eV/atom")


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/mace/calculators/mace.py:143: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_lo

Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (mace-mp-0b3-medium) = -1.371885 eV/atom
Li2O           :  E_form (mace-mp-0b3-medium) = -1.840339 eV/atom
Li7La3Zr2O12   :  E_form (mace-mp-0b3-medium) = -2.815677 eV/atom
ZrO2           :  E_form (mace-mp-0b3-medium) = -3.376098 eV/atom
La2O3          :  E_form (mace-mp-0b3-medium) = -3.492873 eV/atom


/home/mehuldarak/anaconda3/envs/mace/lib/python3.13/site-packages/pymatgen/io/cif.py:1290: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


# MACE T2


In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/summer/mace_models/finetuned/mace_T1_finetune_h200_cn10_compiled.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/serialization.py:1488: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.871396 eV/atom


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -3.770203 eV/atom
Zr: μ_model = -9.089273 eV/atom
O: μ_model = 199.640698 eV/atom


In [ ]:
import os
# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/summer/mace_models/finetuned/mace_T1_finetune_h200_cn10_compiled.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": -1.871396,
    "La": -3.770203,
    "Zr": -9.089273,
    "O":  199.640698,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE_T1) = {e_form: .6f} eV/atom")


Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE_T1) = -94.461143 eV/atom
Li2O           :  E_form (MACE_T1) = -70.527496 eV/atom
Li7La3Zr2O12   :  E_form (MACE_T1) = -105.484956 eV/atom
ZrO2           :  E_form (MACE_T1) = -139.986869 eV/atom
La2O3          :  E_form (MACE_T1) = -126.627813 eV/atom


In [ ]:
/home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen.model


In [ ]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


AttributeError: 'collections.OrderedDict' object has no attribute 'to'

In [ ]:

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    print(f"{label:15s}:  E_total (MACE_T1) = {energy_total: .6f} eV/atom")


Li2O2          :  E_total (MACE_T1) =  35.388065 eV/atom
Li2O           :  E_total (MACE_T1) = -62.738323 eV/atom
Li7La3Zr2O12   :  E_total (MACE_T1) = -1428.315959 eV/atom
ZrO2           :  E_total (MACE_T1) = -119.073932 eV/atom
La2O3          :  E_total (MACE_T1) = -41.757378 eV/atom


# MACE-MP-0b3

In [2]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/mace_universal_models/mace-mp-0b3-medium.model"], device="cpu")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.906338 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -4.895953 eV/atom
Zr: μ_model = -8.559929 eV/atom
O: μ_model = -4.901506 eV/atom


In [ ]:
import os
# ---- 1. Load the MACE model -------------------------------------------
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/mace_universal_models/mace-mp-0b3-medium.model"], device="cpu")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": mu_model_Li,
    "La": mu_model_La,
    "Zr": mu_model_Zr,
    "O":  mu_model_O,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MP-0b3) = {e_form: .6f} eV/atom")


/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)
/home/phanim/harshit

Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE-MP-0b3) = -1.350035 eV/atom
Li2O           :  E_form (MACE-MP-0b3) = -1.821697 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MP-0b3) = -2.794101 eV/atom
ZrO2           :  E_form (MACE-MP-0b3) = -3.348463 eV/atom
La2O3          :  E_form (MACE-MP-0b3) = -3.472036 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


# MACE-MPA-0

In [6]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/mace_universal_models/mace-mpa-0-medium.model"], device="cpu")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.904626 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -4.909520 eV/atom
Zr: μ_model = -8.539511 eV/atom
O: μ_model = -4.931498 eV/atom


In [8]:
import os
# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": mu_model_Li,
    "La": mu_model_La,
    "Zr": mu_model_Zr,
    "O":  mu_model_O,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MPA-0) = {e_form: .6f} eV/atom")


Li2O2          :  E_form (MACE-MPA-0) = -1.374669 eV/atom
Li2O           :  E_form (MACE-MPA-0) = -1.833673 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MPA-0) = -2.789009 eV/atom
ZrO2           :  E_form (MACE-MPA-0) = -3.353263 eV/atom
La2O3          :  E_form (MACE-MPA-0) = -3.458425 eV/atom


# MACE-MATPES_PBE-OMAT-FT

In [9]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/mace_universal_models/mace-mpa-0-medium.model"], device="cpu")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.904626 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -4.909520 eV/atom
Zr: μ_model = -8.539511 eV/atom
O: μ_model = -4.931498 eV/atom


In [10]:
import os
# ---- 2. Reference μ_model from MACE -----------------------------------
# We use Li: μ_model = -1.894110 eV/atom
# La: μ_model = -4.891070 eV/atom
# Zr: μ_model = -8.539969 eV/atom
# O: μ_model = -4.870034 eV/atom
mu_mace = {
    "Li": mu_model_Li,
    "La": mu_model_La,
    "Zr": mu_model_Zr,
    "O":  mu_model_O,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE-MATPES_PBE-OMAT-FT) = {e_form: .6f} eV/atom")


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Li2O2          :  E_form (MACE-MATPES_PBE-OMAT-FT) = -1.374669 eV/atom
Li2O           :  E_form (MACE-MATPES_PBE-OMAT-FT) = -1.833673 eV/atom
Li7La3Zr2O12   :  E_form (MACE-MATPES_PBE-OMAT-FT) = -2.789009 eV/atom
ZrO2           :  E_form (MACE-MATPES_PBE-OMAT-FT) = -3.353263 eV/atom
La2O3          :  E_form (MACE-MATPES_PBE-OMAT-FT) = -3.458425 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


# mace-omat-0-medium

In [11]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/mace_universal_models/mace-omat-0-medium.model"], device="cpu")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")


/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.902394 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = -4.870432 eV/atom
Zr: μ_model = -8.515151 eV/atom
O: μ_model = -4.941141 eV/atom


In [12]:
import os
mu_mace = {
    "Li": mu_model_Li,
    "La": mu_model_La,
    "Zr": mu_model_Zr,
    "O":  mu_model_O,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "./cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = mace_calc
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (mace-omat-0-medium) = {e_form: .6f} eV/atom")


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Li2O2          :  E_form (mace-omat-0-medium) = -1.386963 eV/atom
Li2O           :  E_form (mace-omat-0-medium) = -1.858176 eV/atom
Li7La3Zr2O12   :  E_form (mace-omat-0-medium) = -2.823696 eV/atom
ZrO2           :  E_form (mace-omat-0-medium) = -3.449203 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La2O3          :  E_form (mace-omat-0-medium) = -3.451693 eV/atom


Now we will compute formation energies with MACE-MP-0a large for the test case1-12 structures

In [4]:
import os
import torch
import pandas as pd
from pymatgen.core import Structure
from chgnet.model import CHGNet

# ------------------ 0) Paths & cases ------------------
cif_dir = "/home/phanim/harshitrawat/summer/formntestingenergy"
cases = [f"Case{i}.cif" for i in [1,2,3,4,5,6,7,8,9,10,11,12]]
torch.set_default_dtype(torch.float32)

# ------------------ 1) Load CHGNet on CPU ------------------
print("Loading CHGNet on CPU ...")
model = CHGNet.load()
model.to("cpu")
current_device = "cpu"
print("✓ CHGNet is on CPU.")

# ------------------ 2) Then move to GPU if available ------------------
if torch.cuda.is_available():
    try:
        model.to("cuda")
        current_device = "cuda"
        torch.cuda.empty_cache()
        print("✓ CHGNet moved to GPU.")
    except Exception as e:
        print(f"⚠️ Could not move to GPU, staying on CPU. Reason: {e}")
else:
    print("ⓘ CUDA not available. Running on CPU.")

# ------------------ 3) Elemental chemical potentials (CHGNet μ) ------------------
mu = {"Li": -1.882, "La": -4.894, "Zr": -8.509, "O": -4.913}

# ------------------ 4) Formation energy computation ------------------
print("\n=== Energies (CHGNet) ===\n")
rows = []

for fname in cases:
    path = os.path.join(cif_dir, fname)
    label = os.path.splitext(fname)[0]
    if not os.path.exists(path):
        print(f"{label:10s}: ❌ file missing")
        continue

    struct = Structure.from_file(path)
    comp = struct.composition
    n_atoms = comp.num_atoms

    # --- Reduced formula for f.u. normalization ---
    red_comp, _ = comp.get_reduced_composition_and_factor()
    n_atoms_fu = red_comp.num_atoms  # atoms per formula unit

    # --- CHGNet prediction (per atom energy in eV) ---
    pred = model.predict_structure(struct)
    e_pred_atom = pred["e"]                     # per atom
    e_pred_total = e_pred_atom * n_atoms        # total

    # --- Reference energies ---
    ref_total = sum(comp[el] * mu[str(el)] for el in comp.elements)

    # --- Formation energies ---
    e_form_total = e_pred_total - ref_total
    e_form_per_atom = e_form_total / n_atoms
    e_form_per_fu = e_form_per_atom * n_atoms_fu

    rows.append({
        "case": label,
        "device": current_device,
        "n_atoms": n_atoms,
        "atoms_per_formula_unit": n_atoms_fu,
        "E_total_eV": e_pred_total,
        "E_per_atom_eV": e_pred_atom,
        "E_form_total_eV": e_form_total,
        "E_form_per_atom_eV": e_form_per_atom,
        "E_form_per_formula_unit_eV": e_form_per_fu,
    })

    print(f"{label:10s}: "
          f"E_tot = {e_pred_total: .6f} eV | "
          f"E_atom = {e_pred_atom: .6f} eV/atom | "
          f"E_form = {e_form_total: .6f} eV | "
          f"E_form_atom = {e_form_per_atom: .6f} eV/atom | "
          f"E_form_fu = {e_form_per_fu: .6f} eV/f.u.")

# ------------------ 5) Save results (Excel + JSON) ------------------
if rows:
    out_xlsx = os.path.join(cif_dir, "chgnet_energies_cases_with_fu.xlsx")
    out_json = os.path.join(cif_dir, "chgnet_energies_cases_with_fu.json")
    df = pd.DataFrame(rows)
    df.to_excel(out_xlsx, index=False)
    df.to_json(out_json, orient="records", indent=2)
    print(f"\nSaved: {out_xlsx}\nSaved: {out_json}")


Loading CHGNet on CPU ...
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
✓ CHGNet is on CPU.
✓ CHGNet moved to GPU.

=== Energies (CHGNet) ===

Case1     : E_tot = -2759.997814 eV | E_atom = -4.259256 eV/atom | E_form = -836.517814 eV | E_form_atom = -1.290923 eV/atom | E_form_fu = -69.709818 eV/f.u.
Case2     : E_tot = -2768.102325 eV | E_atom = -4.271763 eV/atom | E_form = -844.622325 eV | E_form_atom = -1.303430 eV/atom | E_form_fu = -70.385194 eV/f.u.
Case3     : E_tot = -2578.770447 eV | E_atom = -4.297951 eV/atom | E_form = -796.382447 eV | E_form_atom = -1.327304 eV/atom | E_form_fu = -199.095612 eV/f.u.
Case4     : E_tot = -2857.218170 eV | E_atom = -3.571523 eV/atom | E_form = -734.726170 eV | E_form_atom = -0.918408 eV/atom | E_form_fu = -183.681543 eV/f.u.
Case5     : E_tot = -2918.403994 eV | E_atom = -3.214101 eV/atom | E_form = -592.655994 eV | E_form_atom = -0.652705 eV/atom | E_form_fu = -148.163998 eV/f.u.
Case6     : E_tot = -2338.025349 eV 

TypeError: array(-4.259256, dtype=float32) (numpy-scalar) is not JSON serializable at the moment

In [6]:
import os
import json
import torch
import pandas as pd
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from mace.calculators import MACECalculator

# ------------------ 0) Paths & cases ------------------
cif_dir = "/home/phanim/harshitrawat/summer/formntestingenergy"
cases = [f"Case{i}.cif" for i in [1,2,3,4,5,6,7,8,9,10,11,12]]
model_path = "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model"

# ------------------ 1) Load MACE on CPU first (with cuEquivariance enabled) ------------------
print("Loading MACE on CPU ...")
mace_calc = MACECalculator(model_paths=[model_path], device="cuda")
print("✓ MACE is on GPU")

# ------------------ 3) Elemental chemical potentials (MACE μ_model) ----
mu_mace = {
    "Li": -1.894110,
    "La": -4.891070,
    "Zr": -8.539969,
    "O":  -4.870034,
}

# ------------------ 4) Compute energies (total/per-atom + formation, per-atom & per-f.u.) ----
rows = []
adaptor = AseAtomsAdaptor()
print("\n=== Energies (MACE) ===\n")

for fname in cases:
    path = os.path.join(cif_dir, fname)
    label = os.path.splitext(fname)[0]
    if not os.path.exists(path):
        print(f"{label:10s}: ❌ file missing")
        continue

    # Load structure
    struct = Structure.from_file(path)
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Atoms per formula unit (reduced composition)
    red_comp, _factor = comp.get_reduced_composition_and_factor()
    n_atoms_per_fu = red_comp.num_atoms  # e.g., Li7La3Zr2O12 -> 24, ZrO2 -> 3

    # Convert to ASE and evaluate with MACE
    ase_atoms = adaptor.get_atoms(struct)
    ase_atoms.calc = mace_calc
    e_total = ase_atoms.get_potential_energy()  # eV (total of the supercell)
    e_per_atom = e_total / n_atoms

    # Reference energies from μ_model (per-atom μ_i × counts in this supercell)
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energies
    e_form_total = e_total - ref_total
    e_form_per_atom = e_form_total / n_atoms
    e_form_per_fu = e_form_per_atom * n_atoms_per_fu  # <- requested per-f.u. convention

    rows.append({
        "case": label,
        "device": current_device,
        "n_atoms": n_atoms,
        "atoms_per_formula_unit": n_atoms_per_fu,
        "E_total_eV": e_total,
        "E_per_atom_eV": e_per_atom,
        "E_form_total_eV": e_form_total,
        "E_form_per_atom_eV": e_form_per_atom,
        "E_form_per_formula_unit_eV": e_form_per_fu,
    })

    print(f"{label:10s} [{current_device}] | "
          f"E_tot = {e_total: .6f} eV | E_atom = {e_per_atom: .6f} eV/atom | "
          f"E_form = {e_form_total: .6f} eV | "
          f"E_form_atom = {e_form_per_atom: .6f} eV/atom | "
          f"E_form_fu = {e_form_per_fu: .6f} eV/f.u.")

# ------------------ 5) Save (Excel + JSON) ------------------------------
if rows:
    out_xlsx = os.path.join(cif_dir, "mace_energies_cases_with_fu.xlsx")
    out_json = os.path.join(cif_dir, "mace_energies_cases_with_fu.json")
    df = pd.DataFrame(rows)
    df.to_excel(out_xlsx, index=False)
    with open(out_json, "w") as f:
        json.dump(rows, f, indent=2)
    print(f"\nSaved: {out_xlsx}\nSaved: {out_json}")


Loading MACE on CPU ...
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
✓ MACE is on GPU

=== Energies (MACE) ===



/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 1 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Case1      [cpu] | E_tot = -2665.710107 eV | E_atom = -4.113750 eV/atom | E_form = -742.402595 eV | E_form_atom = -1.145683 eV/atom | E_form_fu = -61.866883 eV/f.u.
Case2      [cpu] | E_tot = -2674.650998 eV | E_atom = -4.127548 eV/atom | E_form = -751.343486 eV | E_form_atom = -1.159481 eV/atom | E_form_fu = -62.611957 eV/f.u.
Case3      [cpu] | E_tot = -2490.768255 eV | E_atom = -4.151280 eV/atom | E_form = -708.768851 eV | E_form_atom = -1.181281 eV/atom | E_form_fu = -177.192213 eV/f.u.
Case4      [cpu] | E_tot = -2777.636675 eV | E_atom = -3.472046 eV/atom | E_form = -652.610503 eV | E_form_atom = -0.815763 eV/atom | E_form_fu = -163.152626 eV/f.u.
Case5      [cpu] | E_tot = -2804.491067 eV | E_atom = -3.088647 eV/atom | E_form = -474.901015 eV | E_form_atom = -0.523019 eV/atom | E_form_fu = -118.725254 eV/f.u.
Case6      [cpu] | E_tot = -2257.892079 eV | E_atom = -4.212485 eV/atom | E_form = -644.814643 eV | E_form_atom = -1.203012 eV/atom | E_form_fu = -161.203661 eV/f.u.
Case7 

In [5]:
import os
import json
import torch
import pandas as pd
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from mace.calculators import MACECalculator

# ------------------ 0) Paths & cases ------------------
cif_dir = "/home/phanim/harshitrawat/summer/formntestingenergy"
cases = [f"Case{i}.cif" for i in [1,2,3,4,5,6,7,8,9,10,11,12]]
model_path = "/home/phanim/harshitrawat/mace_universal_models/mace-mp-0b3-medium.model"

# ------------------ 1) Load MACE on CPU first (with cuEquivariance enabled) ------------------
print("Loading MACE on CPU ...")
mace_calc = MACECalculator(model_paths=[model_path], device="cpu")
current_device = "cpu"
print("✓ MACE is on CPU (cuEquivariance requested).")

# ------------------ 2) Then move to GPU if available (rebuild) ----------
if torch.cuda.is_available():
    try:
        mace_calc = MACECalculator(model_paths=[model_path], device="cuda", enable_cueq=True)
        current_device = "cuda"
        torch.cuda.empty_cache()
        print("✓ MACE moved to GPU (cuEquivariance requested).")
    except Exception as e:
        print(f"⚠️ Could not move to GPU, staying on CPU. Reason: {e}")
else:
    print("ⓘ CUDA not available. Running on CPU.")

# ------------------ 3) Elemental chemical potentials (MACE μ_model) ----
mu_mace = {
    "Li": -1.906338,
    "La": -4.895953,
    "Zr": -8.559929,
    "O":  -4.901506,
}

# ------------------ 4) Compute energies (total/per-atom + formation, per-atom & per-f.u.) ----
rows = []
adaptor = AseAtomsAdaptor()
print("\n=== Energies (MACE) ===\n")

for fname in cases:
    path = os.path.join(cif_dir, fname)
    label = os.path.splitext(fname)[0]
    if not os.path.exists(path):
        print(f"{label:10s}: ❌ file missing")
        continue

    # Load structure
    struct = Structure.from_file(path)
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Atoms per formula unit (reduced composition)
    red_comp, _factor = comp.get_reduced_composition_and_factor()
    n_atoms_per_fu = red_comp.num_atoms  # e.g., Li7La3Zr2O12 -> 24, ZrO2 -> 3

    # Convert to ASE and evaluate with MACE
    ase_atoms = adaptor.get_atoms(struct)
    ase_atoms.calc = mace_calc
    e_total = ase_atoms.get_potential_energy()  # eV (total of the supercell)
    e_per_atom = e_total / n_atoms

    # Reference energies from μ_model (per-atom μ_i × counts in this supercell)
    ref_total = sum(comp[el] * mu_mace[str(el)] for el in comp.elements)

    # Formation energies
    e_form_total = e_total - ref_total
    e_form_per_atom = e_form_total / n_atoms
    e_form_per_fu = e_form_per_atom * n_atoms_per_fu  # <- requested per-f.u. convention

    rows.append({
        "case": label,
        "device": current_device,
        "n_atoms": n_atoms,
        "atoms_per_formula_unit": n_atoms_per_fu,
        "E_total_eV": e_total,
        "E_per_atom_eV": e_per_atom,
        "E_form_total_eV": e_form_total,
        "E_form_per_atom_eV": e_form_per_atom,
        "E_form_per_formula_unit_eV": e_form_per_fu,
    })

    print(f"{label:10s} [{current_device}] | "
          f"E_tot = {e_total: .6f} eV | E_atom = {e_per_atom: .6f} eV/atom | "
          f"E_form = {e_form_total: .6f} eV | "
          f"E_form_atom = {e_form_per_atom: .6f} eV/atom | "
          f"E_form_fu = {e_form_per_fu: .6f} eV/f.u.")

# ------------------ 5) Save (Excel + JSON) ------------------------------
if rows:
    out_xlsx = os.path.join(cif_dir, "mace_energies_cases_with_fu_03b.xlsx")
    out_json = os.path.join(cif_dir, "mace_energies_cases_with_fu_03b.json")
    df = pd.DataFrame(rows)
    df.to_excel(out_xlsx, index=False)
    with open(out_json, "w") as f:
        json.dump(rows, f, indent=2)
    print(f"\nSaved: {out_xlsx}\nSaved: {out_json}")


Loading MACE on CPU ...


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
✓ MACE is on CPU (cuEquivariance requested).
Using head default out of ['default']
No dtype selected, switching to float64 to match model dtype.
Converting models to CuEq for acceleration


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/calculators/mace.py:197: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/modules/models.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "atomic_numbers", torch.tensor(atomic_numbers, dtype=torch.int64)
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`

⚠️ Could not move to GPU, staying on CPU. Reason: Error(s) in loading state_dict for ScaleShiftMACE:
	Unexpected key(s) in state_dict: "products.0.symmetric_contractions.weight", "products.1.symmetric_contractions.weight". 

=== Energies (MACE) ===

Case1      [cpu] | E_tot = -2669.394191 eV | E_atom = -4.119435 eV/atom | E_form = -735.470651 eV | E_form_atom = -1.134986 eV/atom | E_form_fu = -61.289221 eV/f.u.
Case2      [cpu] | E_tot = -2677.569247 eV | E_atom = -4.132051 eV/atom | E_form = -743.645707 eV | E_form_atom = -1.147601 eV/atom | E_form_fu = -61.970476 eV/f.u.
Case3      [cpu] | E_tot = -2494.365541 eV | E_atom = -4.157276 eV/atom | E_form = -702.521933 eV | E_form_atom = -1.170870 eV/atom | E_form_fu = -175.630483 eV/f.u.
Case4      [cpu] | E_tot = -2783.927634 eV | E_atom = -3.479910 eV/atom | E_form = -646.736230 eV | E_form_atom = -0.808420 eV/atom | E_form_fu = -161.684058 eV/f.u.
Case5      [cpu] | E_tot = -2809.403958 eV | E_atom = -3.094057 eV/atom | E_form = -466.

In [1]:
import os
import shutil

# === CONFIG ===
search_dirs = [
    "/home/phanim/harshitrawat/summer/md/mdcifs",
    "/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed",
    "/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed_prime",
]
output_dir = "/home/phanim/harshitrawat/summer/formntestingenergy"

# Make sure output folder exists
os.makedirs(output_dir, exist_ok=True)

# Case files you gave
case_files = {
    "Case10": "cellrelaxed_LLZO_011_La_code71_sto__Li_100_slab_heavy_T300_0138_strain+0.015_perturbed",
    "Case1": "cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed",
    "Case2": "cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0043_strain-2_perturbed",
    "Case3": "cellrelaxed_LLZO_010_La_order0_off__Li_100_slab_heavy_T450_0089",
    "Case4": "cellrelaxed_LLZO_010_Li_order0_off__Li_110_slab_heavy_T300_0195",
    "Case5": "cellrelaxed_LLZO_010_Li_order0_off__Li_111_slab_heavy_T450_0163_strain-0.010_perturbed",
    "Case6": "cellrelaxed_LLZO_010_Li_order4_off__Li_100_slab_heavy_T450_0046",
    "Case7": "cellrelaxed_LLZO_010_Li_order4_off__Li_111_slab_heavy_T300_0137_strain-2_perturbed",
    "Case8": "cellrelaxed_LLZO_010_O_order5_off__Li_110_slab_heavy_T450_0063",
    "Case9": "cellrelaxed_LLZO_011_La_code71_sto__Li_100_slab_heavy_T300_0039",
    "Case11": "cellrelaxed_LLZO_010_Li_order0_off__Li_111_slab_heavy_T450_0002_strain+3_perturbed",
    "Case12": "cellrelaxed_LLZO_010_Li_order0_off__Li_111_slab_heavy_T450_0002_strain+3_perturbed",
}

# Copy loop
for case, fname in case_files.items():
    found = False
    for src_dir in search_dirs:
        src_path = os.path.join(src_dir, fname + ".cif")
        if os.path.exists(src_path):
            dst_path = os.path.join(output_dir, f"{case}.cif")
            shutil.copy(src_path, dst_path)
            print(f"Copied {src_path} -> {dst_path}")
            found = True
            break
    if not found:
        print(f"❌ File not found in any directory: {fname}.cif")


Copied /home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed_prime/cellrelaxed_LLZO_011_La_code71_sto__Li_100_slab_heavy_T300_0138_strain+0.015_perturbed.cif -> /home/phanim/harshitrawat/summer/formntestingenergy/Case10.cif
Copied /home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed.cif -> /home/phanim/harshitrawat/summer/formntestingenergy/Case1.cif
Copied /home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0043_strain-2_perturbed.cif -> /home/phanim/harshitrawat/summer/formntestingenergy/Case2.cif
Copied /home/phanim/harshitrawat/summer/md/mdcifs/cellrelaxed_LLZO_010_La_order0_off__Li_100_slab_heavy_T450_0089.cif -> /home/phanim/harshitrawat/summer/formntestingenergy/Case3.cif
Copied /home/phanim/harshitrawat/summer/md/mdcifs/cellrelaxed_LLZO_010_Li_order0_off__Li_110_slab_heavy_T300_0195.cif -> /home/phanim/harshitra

In [1]:
# %% Align DFT-FE formation energies to CHGNet / MACE scales
import numpy as np
import pandas as pd

# ---- Per-FU stoichiometries you reported ----
# Case1/2: Li37 La3 Zr2 O12
# Case3:   Li102 La9 Zr5 O34
# Case4:   Li155 La8 Zr5 O32
# Case5:   Li182 La8 Zr5 O32
# Case6:   Li90  La8 Zr5 O31
# Case7:   Li180 La8 Zr5 O31
nu = {
    "Case1": [37, 3, 2, 12],
    "Case2": [37, 3, 2, 12],
    "Case3": [102, 9, 5, 34],
    "Case4": [155, 8, 5, 32],
    "Case5": [182, 8, 5, 32],
    "Case6": [90,  8, 5, 31],
    "Case7": [180, 8, 5, 31],
}

# ---- ML formation energies per FU (eV) you provided ----
CHG = {
    "Case1": -69.7098,  "Case2": -70.3852, "Case3": -199.096,
    "Case4": -183.682,  "Case5": -148.164, "Case6": -181.126,
    "Case7": -134.312,
}
MACE = {
    "Case1": -61.8669,  "Case2": -62.612,  "Case3": -177.192,
    "Case4": -163.153,  "Case5": -118.725, "Case6": -161.204,
    "Case7": -88.7348,
}

# ---- DFT-FE formation energies per FU (eV) computed earlier ----
# (From your DFT totals & Z using μ from elemental supercells; Cases 1–3 and 5–7)
DFT = {
    "Case1":  984.69464303,
    "Case2":  896.30771575,
    "Case3": 2163.84331783,
    # "Case4":  <not provided in earlier DFT batch>  # leave out if unknown
    "Case5": 4485.96568408,
    "Case6": 2011.15980051,
    "Case7": 5303.88489320,
}

# ---- Assemble common cases present in both sets ----
def common_keys(a, b):
    return [k for k in a.keys() if k in b]

cases_chg = common_keys(DFT, CHG)
cases_mace = common_keys(DFT, MACE)

def fit_constant_offset(DFT_dict, MODEL_dict, cases):
    y = np.array([MODEL_dict[k] for k in cases])
    x = np.array([DFT_dict[k] for k in cases])
    # Least-squares: y ≈ x + c  => solve for c
    c = np.mean(y - x)
    y_pred = x + c
    resid = y - y_pred
    return c, resid, y, y_pred

def fit_species_corrections(DFT_dict, MODEL_dict, nu_dict, cases):
    # y ≈ DFT + A * delta_mu; with A rows = [nu_Li, nu_La, nu_Zr, nu_O]
    y = np.array([MODEL_dict[k] for k in cases])
    base = np.array([DFT_dict[k] for k in cases])
    A = np.array([nu_dict[k] for k in cases], dtype=float)  # shape (n, 4)
    # Solve least squares for delta_mu (4x1)
    delta_mu, *_ = np.linalg.lstsq(A, y - base, rcond=None)
    y_pred = base + A @ delta_mu
    resid = y - y_pred
    return delta_mu, resid, y, y_pred

def summarize(name, resid):
    mae = np.mean(np.abs(resid))
    rmse = float(np.sqrt(np.mean(resid**2)))
    return pd.Series({"Model": name, "MAE (eV/FU)": mae, "RMSE (eV/FU)": rmse})

# ---- Fit & report: CHGNet ----
c_chg, r_chg_c, y_chg, yhat_chg_c = fit_constant_offset(DFT, CHG, cases_chg)
dm_chg, r_chg_s, _, yhat_chg_s = fit_species_corrections(DFT, CHG, nu, cases_chg)

# ---- Fit & report: MACE ----
c_mace, r_mace_c, y_mace, yhat_mace_c = fit_constant_offset(DFT, MACE, cases_mace)
dm_mace, r_mace_s, _, yhat_mace_s = fit_species_corrections(DFT, MACE, nu, cases_mace)

# ---- Tables ----
tab_const = pd.DataFrame([
    summarize("CHGNet (const shift)", r_chg_c),
    summarize("MACE (const shift)",   r_mace_c),
])

tab_species = pd.DataFrame([
    summarize("CHGNet (species shifts)", r_chg_s),
    summarize("MACE (species shifts)",   r_mace_s),
])

corr_df = pd.DataFrame({
    "Δμ_Li (eV)": [dm_chg[0], dm_mace[0]],
    "Δμ_La (eV)": [dm_chg[1], dm_mace[1]],
    "Δμ_Zr (eV)": [dm_chg[2], dm_mace[2]],
    "Δμ_O  (eV)": [dm_chg[3], dm_mace[3]],
}, index=["CHGNet", "MACE"])

print("=== Constant-offset alignment (DFT → MODEL) ===")
display(tab_const.round(3))
print(f"CHGNet c = {c_chg:.3f} eV/FU   |   MACE c = {c_mace:.3f} eV/FU\n")

print("=== Species-correction alignment (DFT → MODEL) ===")
display(tab_species.round(3))
print("\nPer-species corrections Δμ (add to DFT formation energy as Σ ν_i Δμ_i):")
display(corr_df.round(3))

# Optional: per-case comparison after alignment
def per_case_table(model_name, cases, y, yhat_c, yhat_s):
    return pd.DataFrame({
        "Case": cases,
        f"{model_name} (target)": y,
        "DFT+const (pred)": yhat_c,
        "DFT+species (pred)": yhat_s,
        "resid const": y - yhat_c,
        "resid species": y - yhat_s,
    })

print("\n--- CHGNet per-case ---")
display(per_case_table("CHGNet", cases_chg, y_chg, yhat_chg_c, yhat_chg_s).round(3))

print("\n--- MACE per-case ---")
display(per_case_table("MACE", cases_mace, y_mace, yhat_mace_c, yhat_mace_s).round(3))


=== Constant-offset alignment (DFT → MODEL) ===


,Model,MAE (eV/FU),RMSE (eV/FU)
0,CHGNet (const shift),1507.592,1697.486
1,MACE (const shift),1497.304,1684.926


CHGNet c = -2774.775 eV/FU   |   MACE c = -2752.698 eV/FU

=== Species-correction alignment (DFT → MODEL) ===


,Model,MAE (eV/FU),RMSE (eV/FU)
0,CHGNet (species shifts),84.357,96.724
1,MACE (species shifts),83.358,95.653



Per-species corrections Δμ (add to DFT formation energy as Σ ν_i Δμ_i):


,Δμ_Li (eV),Δμ_La (eV),Δμ_Zr (eV),Δμ_O (eV)
CHGNet,-34.133,-1935.188,-1623.765,785.340
MACE,-33.870,-1891.606,-1596.988,769.707



--- CHGNet per-case ---


,Case,CHGNet (target),DFT+const (pred),DFT+species (pred),resid const,resid species
0,Case1,-69.710,-1790.080,92.772,1720.370,-162.482
1,Case2,-70.385,-1878.467,4.385,1808.082,-74.771
2,Case3,-199.096,-610.932,-151.645,411.836,-47.451
3,Case5,-148.164,1711.191,-195.615,-1859.355,47.451
4,Case6,-181.126,-763.615,-315.569,582.489,134.443
5,Case7,-134.312,2529.110,-94.770,-2663.422,-39.542



--- MACE per-case ---


,Case,MACE (target),DFT+const (pred),DFT+species (pred),resid const,resid species
0,Case1,-61.867,-1768.004,99.177,1706.137,-161.043
1,Case2,-62.612,-1856.391,10.790,1793.779,-73.402
2,Case3,-177.192,-588.855,-130.303,411.663,-46.889
3,Case5,-118.725,1733.267,-165.614,-1851.992,46.889
4,Case6,-161.204,-741.539,-294.056,580.335,132.852
5,Case7,-88.735,2551.186,-49.661,-2639.921,-39.074
